## Example Use Cases for HTRC's Extracted Features Dataset 

### Introduction
The [Extracted Features (EF) 2.0 dataset](https://wiki.htrc.illinois.edu/display/COM/Extracted+Features+Dataset) contains structured information about each volume in [HathiTrust](www.hathitrust.org). This notebook, and the [accompanying webpage](https://wiki.htrc.illinois.edu/display/COM/EF+Use+Cases+and+Examples), aim to demonstrate example use cases for using the files, and, where possible, making use of the different levels of data contained in the EF Dataset.

If you're new to the dataset, we'd strongly encourage you to start with documentation to get familiar with the data and its structure. For more information about the EF 2.0 Dataset, see the following pages:
* [How to download the dataset](https://wiki.htrc.illinois.edu/display/COM/Downloading+Extracted+Features)
* [Basic Walk through an EF file]
* ["Extracted Features in the Wild"](https://wiki.htrc.illinois.edu/display/COM/Extracted+Features+in+the+Wild): other use cases for the EF Dataset.

Our sample use cases in this notebook are centered around using metadata and features to identify poetry amongst prose. We'll focus on two specific examples:
1. Identifying which volume is poetry and which is prose when given mixed volumes by a single author.
2. Identifying pages of poetry within a single volume that mixes both prose and poetry.

The EF 2.0 Dataset can be used with any programming language, but this page will demonstrate examples using Python. We’ll also be using the [HTRC FeatureReader](https://github.com/htrc/htrc-feature-reader), a Python library with a number of tools for working with the EF 2.0 Dataset, including functionality to download and parse files, as well as display and analyze metadata and features at the page and volume level. For a similar library for working with EF data in R, see Prof. Ben Schmidt’s [hathidy](https://github.com/HumanitiesDataAnalysis/hathidy) library.

### The Data

For our first use case, we'll be using two volumes by Ursula K. Le Guin, who wrote both prose and poetry. Our goal with these volumes will be to try to identify which volume is prose and which poetry using the Extracted Features Dataset.

For our second use case, we'll be using volume 142 of an anthology of *Harper's Magazine* covering issues from 1920 and 1921. This particular volume is verified as having poetry within it, including a relatively famous poem by Robert Frost (but maybe not the one you're thinking of!).

### Use Case 1: Using page-level features to identify poetry and prose in different volumes
Before running an algorithm for text analysis, it’s important to understand the data being studied. For traditional research that utilizes tens of books, this step is covered when reading the material directly. For computational analysis, where often the focus are sets of hundreds or thousands of books (what HTRC calls “worksets”), this is not achieved so easily, and understanding and exploring our data becomes an explicit step in the research process. Understanding the data in our workset is the first step to computational analysis, but often can present a number of challenges. For instance, If we were interested in studying poetry through traditional means, we could simply visit our local library, go to the poetry section and start reading. If we wanted to see if there are volumes that are poetry but not included in this section, we could open them and see if we find poetry or prose. But when working with hundreds and thousands of digital items, we do not have the ability to practically do either of these things. Some volumes we want to study may be under copyright, which means we can’t read them without purchasing. Others may have incorrect metadata records attached to them, which means we may not even know to look for them as poetry. Perhaps most challenging of all, it would take lifetimes to manually read thousands of books.

This is where the Extracted Features 2.0 Dataset can come in handy. Since the dataset is [non-consumptive](https://www.hathitrust.org/htrc_ncup), we can use the data, without restriction, even to study volumes we cannot read due to copyright. Additionally, the dataset is heavily structured, removing the need for users to replicate common data analysis tasks like part-of-speech tagging, or page section separation. This structure also lets us quickly explore thousands of volumes, and do so relatively quickly.

Returning to our first example, suppose we are a scholar interested in studying large-scale trends in poetry. Our first step would be to assemble a workset of poetry and then use it to answer our research questions. With a massive digital library like the HathiTrust, finding all of the items we are interested in is no easy task. However, we can use the EF 2.0 Dataset to assess if there are structural characteristics of texts that we know are correctly classified as poetry, and potentially develop a generalized method of identifying poetry based on these characteristics. We’ll be doing exactly this using two unidentified volumes by Ursula K. Le Guin, who published both prose and poetry.

To start, we’ll use each of our volume’s HathiTrust IDs, their permanent, unique identifier in HathiTrust, to download their EF files and create FeatureReader Volume objects--which allow us to take advantage of volume-level data in the files--for each of our volumes of interest. A Volume in FeatureReader has specific properties and built-in methods that let us do a number of common tasks. We can create Volumes simply by using Volume() and adding the HathiTrust ID (HTID) in the parentheses:

As with most Jupyter notebooks, we'll start by importing some libraries we'll need to tackle these examples, [Pandas](https://pandas.pydata.org/), a very common data science library in Python, and the HTRC FeatureReader, a library written by HTRC to ease the use of the Extracted Features Dataset:

In [ ]:
from htrc_features import FeatureReader, Volume
import pandas as pd

Next, we'll create FeatureReader Volume objects for each of our volumes of interest. A Volume in FeatureReader has specific properties and built-in methods that let us do a number of common tasks. We can create Volumes simply by using `Volume()` and adding the HathiTrust ID (HTID) in the parentheses:

In [ ]:
leguin_v1 = Volume('mdp.39015000639800') 
print(f"{leguin_v1.author}, {leguin_v1.pub_date}, {leguin_v1.id}")

leguin_v2 = Volume('mdp.39015052467530')
print(f"{leguin_v2.author}, {leguin_v2.pub_date}, {leguin_v2.id}")

After running the above code, the print statements will return and we’ll have Volume objects for each work by Le Guin. For this example, we are using the FeatureReader to download the files into the memory of our Python session. This works well for small worksets, but we should only use this method for <100 volumes, or else it will become less efficient and practical.

We'll be focusing on one particular field of features in the EF Dataset: begin-line characters. This is because we’re working off of the assumption that a page of poetry is likely to contain more capitalized letters at the start of each line on a page, a common poetry convention Le Guin is known to have used. To start analyzing the structure of our volumes, we'll create a Pandas [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) with each begin line character, by page, for each volume:

In [ ]:
leguin_v1_blc = leguin_v1.begin_line_chars() 
leguin_v2_blc = leguin_v2.begin_line_chars()

leguin_v1_blc.head()
# leguin_v2_blc.head()

With this DataFrame, we have every begin-line character (BLC, for brevity) from our volumes, their counts by page, which section of the page they occur in (header, body, footer) and the page sequence number on which they occur. FeatureReader makes getting this data very easy (only 2 lines of code!), and retrieves it in a format that lets us go through and make our own counts with just a simple `for` loop:

In [ ]:
leguin_v1_blc_up = 0 # a variable that will count uppercase BLCs
leguin_v1_blc_low = 0 # a variable that will count lowercase BLCs


# iterate through rows of our BLC DataFrame
for idx, count in leguin_v1_blc.iterrows():
    blc = idx[3] # assign third index, the character, to variable
    if blc.isupper() == True: # check if the BLC is uppercase
            leguin_v1_blc_up +=1 # if uppercase, add 1 to uppercase BLC
    else:
            leguin_v1_blc_low +=1 # if not, add 1 to lowercase BLC

# The next section has print statements that will report our results
print(f"Found {leguin_v1_blc_up} capitalized begin line characters")
print(f"Found {leguin_v1_blc_low} lowercase begin line characters")
print(f"Found {leguin_v1_blc_up + leguin_v1_blc_low} total begin line characters")


# using counts to generate percentage
total_char = len(leguin_v1_blc['count'])
leguin_v1_caps_pct = leguin_v1_blc_up / total_char

# printing results, with rounded percentage
print(f"Uppercase characters make up {'{:.2%}'.format(leguin_v1_caps_pct)} of begin line characters")

Ok, so we found that over 50% of all BLC in Volume 1 by Le Guin are capitalized. That seems like a substantial amount, but it's hard to know for sure without comparison to our other volume. Let's repeat the code for Volume 2 and see what we find:

In [ ]:
# code in this cell is identical to the code above, save for changed labels
leguin_v2_blc_up = 0
leguin_v2_blc_low = 0

for idx, count in leguin_v2_blc.iterrows():
    # print(count)
    blc = idx[3]
    # print(blc)
    if blc.isupper() == True:
            leguin_v2_blc_up +=1
    else:
            leguin_v2_blc_low +=1

print(f"Found {leguin_v2_blc_up} capitalized begin line characters")
print(f"Found {leguin_v2_blc_low} lowercase begin line characters")
print(f"Found {leguin_v2_blc_up + leguin_v2_blc_low} total begin line characters")
        
total_char = len(leguin_v2_blc['count'])  
leguin_v2_caps_pct = leguin_v2_blc_up / total_char
print(f"Uppercase characters make up {'{:.2%}'.format(leguin_v2_caps_pct)} of begin line characters")

From over 50% capitalization of BLC to 23.5% is a steep reduction. It now seems likely that our first volume is poetry and the second likely to be prose. This process is still part of our exploratory data analysis phase, so we should do our best to verify these results, if possible. Since both volumes came from HathiTrust, we have bibliographic metadata for each volume, and can access certain fields within FeatureReader. Let's see what we can find out by retrieving the titles of our mystery volumes, along with a link back to the item in HathiTrust, in case we want to take a closer look:

In [ ]:
print(f"leguin_v1: {leguin_v1.title}, {leguin_v1.ht_bib_url}")

print(f"leguin_v2: {leguin_v2.title}, {leguin_v2.ht_bib_url}")

Success! `leguin_v1` makes verification easy by including "and other poems" in its title. We could also use the returned URL to examine the volumes' bibliographic record and see if there is a marker for poetry. (If these volumes were in the public domain, we could also get their handle URLs using `leguin_v1.handle_url` and examine the texts in the HathiTrust Digital Library.)

### Example 2: Finding poetry within a mixed volume
We've shown above that, given two volumes from a single author, we can use the page-level metadata in  Extracted Features (EF) 2.0 Dataset files to identify which is poetry and which is prose. But what about a single volume with mixed poetry and prose included? A similar workflow should be successful, but let's try.

In this example we'll generate a benchmark value of tokens-per-page, based on the mean number of tokens per page in volume 142 of an anthology of *Harper's Magazine* covering issues from 1920 and 1921. We’ll then replicate the begin line characters analysis we did for Case Study 1, above, but modified slightly to delineate pages that are suspected poetry from suspected prose. Pages that appear as suspected poetry by both metrics will be our results, which we'll then do our best to manually verify.

But first, we'll import the Python libraries we need, and then we'll load our volume of Harper’s as a FeatureReader Volume:

In [ ]:
from htrc_features import FeatureReader, Volume
import pandas as pd


# creating a FeatureReader Volume from our issue of Harper's
harpers = Volume('coo.31924054824473')

# printing info about the volume, available via FeatureReader methods
print(f"{harpers.title}, {harpers.enumeration_chronology}, {harpers.id}")

We now have a FeatureReader Volume from our anthology of *Harper's*, and we can take advantage of the methods that the FeatureReader includes, one of which is `tokens_per_page()` which will return a sparse DataFrame with (you guessed it) tokens by page. This DataFrame will also allow us to identify the mean number of tokens per page in this volume, which we'll use for our benchmarking later on:

In [ ]:
# create a new DataFrame from our Volume, using tokens_per_page()
h_tokens = harpers.tokens_per_page()

# assign the mean of tokens per page in the volume to new variable
h_avg_token_ct = h_tokens.mean() 

# printing our mean
print(f"Average tokens per page in this volume: {round(h_avg_token_ct)}")
# looking at the first 15 rows of our new DataFrame
h_tokens.head()

Next, we'll use the mean per-page token count to calculate a benchmark figure of one standard deviation below the mean that we will use to identify suspected pages of poetry. Pandas allows you to get this number very easily for a given column (what Pandas calls a "series") of numerical data using `.std()`. We'll then subtract the standard deviation from the mean to get a benchmark number of tokens that we will use to compare each page of the volume:

In [ ]:
# calculate the standard deviation of data in token counts by page
h_tokens.std() 
print(h_tokens.std())

# create a benchmark 1 standard deviation below mean token count per page:
benchmark = (h_avg_token_ct - h_tokens.std())
print(benchmark)

The benchmark number of tokens we’ll be using is ~377 for a given page. However, this is imprecise and our project is exploratory, which means we don’t yet really know what the most accurate benchmark for poetry in this volume is. As a result, we will be considering pages that have a total number of tokens within 100 of this number, above or below, as possible pages of poetry. This will cast a wide net in the hopes of not missing any possible pages of poetry. This works for our task because (as you’ll see later) we are also considering begin line characters in conjunction with tokens per page, which helps reduce the wide net to the most likely candidate pages. For any given project, these types of decisions will need to be made by the researcher given their preferences, understanding of the data, and chosen methodological approach.

In the next code cell, we'll be using our benchmark token count to gather a preliminary list of suspected poetry pages by iterating through our tokens-per-page DataFrame `h_tokens` and recording rows where the token count is within 100 tokens of our benchmark:

In [ ]:
poetry_pages = [] # a list of pag numbers of suspected poetry
other_pages = [] # a list of page numbers of suspected non-poetry

# iterate through rows of DataFrame
for page, count in h_tokens.iteritems():
    # if count in range of benchmark, add page to list of possible poetry
    if (benchmark - 100) <= count <= (benchmark + 100):
        poetry_pages.append(page)
    
      # if not in range, add to other list      
    else:
        other_pages.append(page)

# print out the number of pages suspected of poetry and those not       
print(f"Found {len(poetry_pages)} suspected pages of poetry")
print(f"Found {len(other_pages)} other pages")

The code above creates first-pass lists of pages that contain possible poetry and possible prose, and adds the page numbers to the appropriate list. We've identified these pages by using a benchmark that’s really just an estimate, and should only be used to identify poetry in conjunction with other analysis or as a first-pass method for weeding out pages that are unlikely to contain traditionally-structured poetry. This method alone is not enough for our use case, so we must combine it with further analysis. We'll do this by looking at the percentage of begin-line characters (letters) are capitalized, and see if we can find a trend.

We'll be re-using the general code and workflow from Case Study 1, where we looked at begin-line characters (BLCs) with some modifications:

In [ ]:
h_blc = harpers.begin_line_chars() # make new DataFrame of BLC for issue of Harper's
h_blc = h_blc.reset_index() # flatten DataFrame for clarity in code

h_blc.head() # look at first 5 rows of new DataFrame

We can iterate through this BLC DataFrame and track how many characters are uppercase, lowercase, and non-alphabetical on each page. This is different to our first example use case, as we are interested in both the numbers of BLCs in each category and the page on which they appear. Otherwise, this is a very similar workflow to what we did in the first example. The big difference is that we will be creating and working with [Python dictionaries](https://www.w3schools.com/python/python_dictionaries.asp) rather than lists. Unlike lists, dictionaries store `key:value` pairs, which means we can store multiple data points within a single entry in the dictionary. Rather than storing one list of page numbers and one list of BLC counts for each page, we can use a single dictionary to store both. This allows us to track BLC types by page and organize them for easy comparison later on. 

In [ ]:
h_blc_up = {} # new dictionary for uppercase BLCs
h_blc_low = {} # new dictionary for lowercase BLCs
h_blc_nonalpha = {} # new dictionary for non-alpha BLCs

# iterate through DataFrame, return indices and row
for i, r in h_blc.iterrows():
    page = r[0] # assign each column of DataFrame to clear variables
    section = r[1]
    token_place = r[2]
    char = r[3]
    count = r[4]
    # use Python method to check if char is alphabetic. If so, continue down this loop:
    if char.isalpha() == True:
        # if BLC is upper, continue down loop:
        if char.isupper() == True:
            # check if dictionary entry for current page exists
            if page not in h_blc_up:
                h_blc_up[page] = 1 # add new dictionary entry for page
            else:
                h_blc_up[page] += 1 # update existing dictionary entry
    
        # if BLC is not upper, continue down this loop:
        else: 
            if page not in h_blc_low:
                h_blc_low[page] = 1
            else:
                h_blc_low[page] += 1
    # if BLC not alphabetic, continue down this loop:
    else:
        if page not in h_blc_nonalpha:
            h_blc_nonalpha[page] = 1
        else:
            h_blc_nonalpha[page] += 1

# printing results            
print(f"Found {len(h_blc_up)} uppercase BLC")
print(f"Found {len(h_blc_low)} lowercase BLC")
print(f"Found {len(h_blc_nonalpha)} non-alpha BLC")

We now have three different dictionaries that contain BLC counts by type for each page. Since they share keys, we can make a new DataFrame with each page sequence as a row, and values for each type of begin line character as columns. This will let us compare BLCs of all  three types, by page, in a single DataFrame. To do this, we’ll create a DataFrame from the dictionaries, rename the columns and transpose its axes to move pages to rows (to allow our code to iteratively search it to be simpler), and then fill all `NaN` (Not a Number) values, which are a result from pages with no lower/upper/non-alpha begin line characters, with 0 in order to compare them numerically to the other values:

In [ ]:
# create a list of dictionaries to convert to DataFrame
dictionaries = [h_blc_nonalpha, h_blc_low, h_blc_up]

# create DataFrame using list of dictionaries
harpers_blc_df = pd.DataFrame.from_dict(dictionaries)
# harpers_blc_df.head(15)

# rename our columns to represent source dictionaries
harpers_blc_df = harpers_blc_df.rename(index={0: "non-alpha", 1: "lowercase", 2: "uppercase"})

# transpose DataFrame to move pages from columns to rows
harpers_blc_df = harpers_blc_df.T

# fill NaN values with 0 in order to compare to other integers
harpers_blc_df = harpers_blc_df.fillna(0)

# look at first 15 rows of modified DataFrame
harpers_blc_df.head()
# harpers_blc_df.shape

Now we can go through the rows and track which category of begin line characters (BLCs) is the most common on each page, adding the page number to a list of possible pages containing poetry, as appropriate. For this example, we're only interested in pages of possible poetry, so we won’t track pages with more lowercase or non-alpha BLCs, only those where uppercase characters are most common. This could easily be adjusted in our code by adding lists for the other two categories of BLCs, and adding `.append()` statements under each loop.

In [ ]:
# create list for tracking page numbers of suspected poetry
blc_poss_poetry = []

# iterate through our BLC DataFrame; return index and row
for i, r in harpers_blc_df.iterrows():
    non_alpha = r[0] # assign parts of row to clear variables
    low = r[1]
    up = r[2]
    # check if non-alpha category has highest token count for page
    if non_alpha > low and non_alpha > up:
        continue 
    # check if lowercase category has highest token count for page
    elif low > non_alpha and low > up:
        continue
    # check if uppercase category has highest token count for page
    elif up > non_alpha and up > low:
        # add page number to list of possible poetry pages
        blc_poss_poetry.append(i)

# verify results by looking at first 15 entries in list of possible poetry
blc_poss_poetry[:15]

With two lists of potential pages of poetry--one based on token counts by page (`poetry_pages`) and one based on capital letters making up the plurality of begin line characters (`blc_poss_poetry`)--we can look for page numbers that appear in both lists. These pages are the most likely candidates to be poetry instead of prose, as they’ve been flagged by their relatively low number of tokens and high number of capitalized BLCs. We can find page numbers in both lists by using one simple piece of code:

In [ ]:
# set ensures all values in poetry_pages list are unique & .intersection() will find values in poetry_pages
#  that also appear in blc_poss_poetry
poetry = set(poetry_pages).intersection(blc_poss_poetry)

# return items in our final list of result pages
poetry

From 990 pages in our volume of Harper’s Magazine, we now have 17 pages that are strong candidates to contain poetry! However, it's important to be reminded (yes, again!) that this use case is showing an exploratory workflow, so it’s best to verify our results, and if applicable, improve our process and code to maximize accuracy based on our data.

This anthology of *Harper's* is in the public domain, so we can directly view the pages we expect to be poetry and see what we find. To make this easier to do in our Jupyter notebook, we'll use the URL to the first page of the volume in HathiTrust's page turner as a base (https://babel.hathitrust.org/cgi/pt?id=coo.31924054824473&view=1up&seq=), and then add a sequence number at the end with our possible page numbers in order to print URLs for each page:

In [ ]:
# assign a root URL to use to generate URLs for each page
harpers_url = 'https://babel.hathitrust.org/cgi/pt?id=coo.31924054824473&view=1up&seq='

# loop through list of pages and add the page number to the end of our root URL
for page in sorted(poetry):
    page_url = harpers_url + str(page)
    print(page_url)

Our results aren’t too bad! There are some false positives, as there are pictures and ads in *Harper's* which can skew our method, though these are relatively unique features in comparison with most volumes. But, we did find valid poetry, including the Robert Frost poem “Fire and Ice” on page 79 (https://babel.hathitrust.org/cgi/pt?id=coo.31924054824473&view=1up&seq=79). These results also help to remind us that any general text analysis workflow will require customization based on knowledge of our target data--both what we hope to find and the pool of data in which we hope to find it!